<a href="https://colab.research.google.com/github/shubh586/Webnews/blob/main/Fontrecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhruvmak/font-recognition-data")

print("Path to dataset files:", path)

100%|██████████| 1.27G/1.27G [01:02<00:00, 21.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dhruvmak/font-recognition-data/versions/1


In [2]:
import shutil
import os

# Corrected path string
original_path = "/root/.cache/kagglehub/datasets/dhruvmak/font-recognition-data/versions/1"
custom_path = "./my_folder"

# Create if not exists, and copy contents
shutil.copytree(original_path, custom_path, dirs_exist_ok=True)

print("Dataset moved to:", custom_path)


Dataset moved to: ./my_folder


In [3]:
datadir=os.path.join("./my_folder/Font Dataset Large")
print(datadir)

./my_folder/Font Dataset Large


In [4]:
for classes in os.listdir(datadir):
    print(classes)

Garamond
Frutiger
Snowdrift Regular
Cambria
Franklin Gothic
Agency
Nasalization
Monotype Corsiva
Courier
Futura
Minion
Lucida Bright
Algerian
Elephant
Didot
Mrs Eaves
Calibry
Fascinate
Calvin
Californian FB
Consolas
Bell MT
Baskerville
Rockwell
Arial
Gill sans
Corbel
LCD Mono
Georgia
Sabon
Brandish
Calligraphy
Bodoni
Myriad
Century
Helvetica
Akzidenz Grotesk
Comic Sans MS
Hombre
Verdana
News Gothic
Palatino linotype
Bembo
Candara
Steppes TT
Book Antiqua
Perpetua
Times New Roman


In [5]:
subsetdir = './fonts'
os.makedirs(subsetdir, exist_ok=True)


In [6]:
for classes in os.listdir(datadir):
    src=os.path.join(datadir, classes)
    dst=os.path.join(subsetdir,classes)
    os.makedirs(dst,exist_ok=True)
    for img in os.listdir(src):
      src1=os.path.join(src,img)
      dest2=os.path.join(dst,img)
      shutil.copy(src1,dest2)


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
traindata="./train"
testdata="./test"
os.makedirs(traindata,exist_ok=True)
os.makedirs(testdata,exist_ok=True)

In [9]:
fontssrc="./fonts"
testdata="./test"
traindata="./train"
for classes in os.listdir(fontssrc):
  # print(classes)
  src=os.path.join(fontssrc,classes)
  if not os.path.isdir(src):
        continue


  # for img in os.listdir(src):
  #   #print(img)
  imgs=os.listdir(src)
  train,test=train_test_split(imgs,test_size=0.2,random_state=42)
  #print(test[:10])
  for img in train:
    src1=os.path.join(src,img)
    dest2=os.path.join(traindata,classes,img)
    os.makedirs(os.path.dirname(dest2),exist_ok=True)
    shutil.copy(src1,dest2)
  for img in test:
        src1 = os.path.join(src, img)
        dest2 = os.path.join(testdata, classes, img)
        os.makedirs(os.path.dirname(dest2), exist_ok=True)
        shutil.copy(src1, dest2)



In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [21]:

IMG_SIZE = (64, 64)
BATCH_SIZE = 64
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    './train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    './test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 192000 images belonging to 48 classes.
Found 48000 images belonging to 48 classes.


In [12]:
train_data.num_classes

48

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(64,64,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Dense(64, activation='relu'),
    BatchNormalization(),

    Dense(32, activation='relu'),
    BatchNormalization(),

    Dense(train_data.num_classes, activation='softmax')
])


In [18]:

model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [19]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 256)    │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 4, 4, 32)       │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 4, 4, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 2, 2, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             

 Total params: 462,896 (1.77 MB)

 Trainable params: 461,360 (1.76 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [22]:

model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)



Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 202s 64ms/step - accuracy: 0.9458 - loss: 0.1884 - val_accuracy: 0.9086 - val_loss: 0.3123
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 151s 50ms/step - accuracy: 0.9767 - loss: 0.0781 - val_accuracy: 0.9086 - val_loss: 0.3001
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 151s 50ms/step - accuracy: 0.9840 - loss: 0.0550 - val_accuracy: 0.9790 - val_loss: 0.0679
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 150s 50ms/step - accuracy: 0.9874 - loss: 0.0405 - val_accuracy: 0.9943 - val_loss: 0.0178
Epoch 5/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 150s 50ms/step - accuracy: 0.9903 - loss: 0.0311 - val_accuracy: 0.9291 - val_loss: 0.2695
Epoch 6/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 152s 50ms/step - accuracy: 0.9916 - loss: 0.0267 - val_accuracy: 0.9933 - val_loss: 0.0189
Epoch 7/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 150s 50ms/step - accuracy: 0.9926 - loss: 0.0229 - val_accuracy: 0.9954 - val_loss: 0.0145
Epoch 8/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 152s 51ms/step - accuracy: 

In [23]:

loss, acc = model.evaluate(test_data)
print(f"Test Accuracy: {acc * 100:.2f}%")


model.save("font_recognition_model_bn_dropout.h5")

750/750 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.9939 - loss: 0.0198


Test Accuracy: 99.37%


In [25]:
from google.colab import files
files.download('font_recognition_model_bn_dropout.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Create label map from train_data
label_map = {v: k for k, v in train_data.class_indices.items()}

def predict_font(img_path, model):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    pred = model.predict(img_array)
    pred_index = np.argmax(pred)

    # Get label
    predicted_font = label_map[pred_index]

    print(f"Predicted font: {predicted_font}")
    return predicted_font



In [34]:
predict_font("/content/screenshot-20250619-214527.png",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted font: Corbel


'Corbel'

In [36]:
predict_font("/content/test/Corbel/Image_10.jpg",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted font: Corbel


'Corbel'

In [41]:
predict_font("/content/arial2.png",model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted font: Candara


'Candara'

In [33]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful